In [1]:
import joblib
import numpy as np
import pandas as pd
from itertools import product
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV
import first_analysis

C:\Users\User\AppData\Local\Temp\ipykernel_49272\1034803330.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def random_forest_cv(X_train, y_train, X_val, y_val, num_fold = 10):
    param_grid = {
        'n_estimators': [100, 150, 200, 250, 300, 350],
        'max_depth': list(range(20, 200, 20)),
        'min_samples_split': list(range(10, 30, 5)),
        'max_features': ['sqrt', 'log2', None]
    }
    
    random_forest = RandomForestRegressor(random_state=42)
    
    grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=num_fold, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    
    
    predizioni_val = best_model.predict(X_val)
    
    mse_val = mean_squared_error(y_val, predizioni_val)
    mae_val = mean_absolute_error(y_val, predizioni_val)
    r2_val = r2_score(y_val, predizioni_val)
    mape_val = mean_absolute_percentage_error(y_val, predizioni_val)
    
    return best_model, grid_search.best_params_, mse_val, mae_val, r2_val, mape_val

In [2]:
# Testa solo con PCA e no outliers --> perchè altrimenti troppe colonne
X_train_PCA = np.load("./Datasets/Train_Val_Test/PCA/no_outliers/principals_components_train.npy")
X_val_PCA = np.load("./Datasets/Train_Val_Test/PCA/no_outliers/principals_components_val.npy")
y_train, y_val = first_analysis.load_df("./Datasets/Train_Val_Test/y_train.csv", "./Datasets/Train_Val_Test/y_val.csv")

In [5]:
X_train_PCA = np.load("./Datasets/Train_Val_Test/PCA/principals_components_train.npy")


In [6]:
len(X_train_PCA)

151273

In [ ]:
best_model, bestparams, mse, mae, r2, mape = random_forest_cv(X_train_PCA, y_train, X_val_PCA, y_val)

In [ ]:
joblib.dump(best_model, './models/random_forest.save')